In [1]:
import os

In [2]:
%pwd

'c:\\Users\\salman.ahmad1.PTCL\\OneDrive - Pakistan Telecomunications Company Limited\\Desktop\\Python Scripts\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\salman.ahmad1.PTCL\\OneDrive - Pakistan Telecomunications Company Limited\\Desktop\\Python Scripts\\Text-Summarizer-Project'

In [7]:
# ----------------------------------------------------------------
# Entity for Data Ingestion Configuration
# ----------------------------------------------------------------
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [18]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [22]:
# ----------------------------------------------------------------
# ---------------------Configuration Manager----------------------
# ----------------------------------------------------------------
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [23]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [24]:
# ----------------------------------------------------------------
# -------------------------Components-----------------------------
# ----------------------------------------------------------------

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file 
            )
            logger.info(f"{filename} downloaded with following information: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into data directory
        Function returns nothing
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [25]:
# ----------------------------------------------------------------
# -------------------------Pipeline-------------------------------
# ----------------------------------------------------------------

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[ 2023-07-23 14:07:51,353: INFO - textSummarizerLogger : lineno-28 - common - yaml file: config\config.yaml loaded successfully]
[ 2023-07-23 14:07:51,373: INFO - textSummarizerLogger : lineno-28 - common - yaml file: params.yaml loaded successfully]
[ 2023-07-23 14:07:51,374: INFO - textSummarizerLogger : lineno-46 - common - created directory at: artifacts]
[ 2023-07-23 14:07:51,377: INFO - textSummarizerLogger : lineno-46 - common - created directory at: artifacts/data_ingestion]


[ 2023-07-23 14:07:53,890: INFO - textSummarizerLogger : lineno-15 - 3888653334 - artifacts/data_ingestion/data.zip downloaded with following information: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1CA8:355447:1DE988:2A3AEE:64BCEDE8
Accept-Ranges: bytes
Date: Sun, 23 Jul 2023 09:07:53 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990020-FJR
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1690103272.421961,VS0,VE822
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: ce9b22cc07bb534b6c9b93ae299ac08404aba15e
Expires: Sun, 23 Jul 2023 09:12:53 GMT
Source-Age: 0